# Scale-Free Graph Conjectures with TxGraffiti

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RandyRDavila/AI-discovery-in-mathematics-with-TxGraffiti/blob/main/notebooks/scale_free_graphs.ipynb)

## Introduction

This notebook demonstrates the application of the TxGraffiti algorithm to generate conjectures on scale-free graphs. Scale-free graphs are a type of graph characterized by the presence of hub nodes with a high degree of connectivity, and they are commonly found in natural and man-made systems, such as social networks and the internet.

## Dataset

The dataset consists of randomly generated scale-free graphs and includes various numerical properties such as:
- **Average Clustering Coefficient**: A measure of the degree to which nodes in a graph tend to cluster together.
- **Degree Assortativity Coefficient**: A measure of the similarity of connections in the graph with respect to the node degree.
- **Number of Triangles**: The number of triangles in the graph, indicating the presence of closely knit groups of nodes.
- **Diameter**: The longest shortest path between any two nodes in the graph.

## Objectives

- Generate conjectures relating different numerical properties of scale-free graphs.
- Identify significant relationships and patterns in graph properties.
- Apply the Theo and Static Dalmatian heuristics to filter and refine the conjectures.

## Usage

1. **Run the cells to generate the dataset and apply TxGraffiti.**
2. **Examine the generated conjectures and their significance.**

Explore the unique properties of scale-free graphs and discover new insights with TxGraffiti.


In [3]:
import pandas as pd
import numpy as np
import networkx as nx
from pulp import *
from fractions import Fraction
from itertools import combinations
import matplotlib.pyplot as plt

# Define the hypothesis, conclusion, and conjecture classes
class Hypothesis:
    def __init__(self, statements):
        self.statements = statements

class LinearConclusion:
    def __init__(self, target, inequality, slope, other, intercept):
        self.target = target
        self.inequality = inequality
        self.slope = slope
        self.other = other
        self.intercept = intercept

class LinearConjecture:
    def __init__(self, hypothesis, conclusion, symbol, touch, type="scale-free graph"):
        self.hypothesis = hypothesis
        self.conclusion = conclusion
        self.symbol = symbol
        self.touch = touch
        self.type = type

    def __repr__(self):
        if self.hypothesis.statements:
            hypothesis_str = " and ".join([f"{self.symbol} is {h}" for h in self.hypothesis.statements])
            return (f"For any {self.type} {self.symbol}, if {hypothesis_str}, then "
                    f"{self.conclusion.target}({self.symbol}) {self.conclusion.inequality} "
                    f"{self.conclusion.slope}*{self.conclusion.other}({self.symbol}) + "
                    f"{self.conclusion.intercept}, with equality on {self.touch} instances.")
        else:
            return (f"For any {self.type} {self.symbol}, "
                    f"{self.conclusion.target}({self.symbol}) {self.conclusion.inequality} "
                    f"{self.conclusion.slope}*{self.conclusion.other}({self.symbol}) + "
                    f"{self.conclusion.intercept}, with equality on {self.touch} instances.")

    def get_sharp_objects(self, df):
        X = df[self.conclusion.other].to_numpy()
        Y = df[self.conclusion.target].to_numpy()
        sharp_indices = df[np.isclose(Y, float(self.conclusion.slope) * X + float(self.conclusion.intercept))].index
        return df.loc[sharp_indices]

    def calculate_distances(self, df):
        X = df[self.conclusion.other].to_numpy()
        Y = df[self.conclusion.target].to_numpy()
        distances = np.abs(Y - (float(self.conclusion.slope) * X + float(self.conclusion.intercept)))
        return distances

def make_upper_linear_conjecture(df, target, other, hypothesis, symbol="G"):
    for hyp in hypothesis:
        df = df[df[hyp] == True]
    X = df[other].to_numpy()
    Y = df[target].to_numpy()

    prob = LpProblem("UpperBoundConjecture", LpMinimize)
    w = LpVariable("w")
    b = LpVariable("b")

    prob += lpSum([w * x + b - y for x, y in zip(X, Y)])

    for x, y in zip(X, Y):
        prob += w * x + b - y >= 0

    prob.solve()

    if w.varValue is None or b.varValue is None:
        return None

    m = Fraction(w.varValue).limit_denominator(10)
    b = Fraction(b.varValue).limit_denominator(10)
    if m == 0:
        return None  # Skip trivial conjectures

    touch = np.sum(np.isclose(Y, float(m) * X + float(b)))

    hypothesis = Hypothesis(hypothesis)
    conclusion = LinearConclusion(target, "<=", m, other, b)

    return LinearConjecture(hypothesis, conclusion, symbol, touch)

def make_lower_linear_conjecture(df, target, other, hypothesis, symbol="G"):
    for hyp in hypothesis:
        df = df[df[hyp] == True]
    X = df[other].to_numpy()
    Y = df[target].to_numpy()

    prob = LpProblem("LowerBoundConjecture", LpMaximize)
    w = LpVariable("w")
    b = LpVariable("b")

    prob += lpSum([w * x + b - y for x, y in zip(X, Y)])

    for x, y in zip(X, Y):
        prob += w * x + b - y <= 0

    prob.solve()

    if w.varValue is None or b.varValue is None:
        return None

    m = Fraction(w.varValue).limit_denominator(10)
    b = Fraction(b.varValue).limit_denominator(10)
    if m == 0:
        return None  # Skip trivial conjectures

    touch = np.sum(np.isclose(Y, float(m) * X + float(b)))

    hypothesis = Hypothesis(hypothesis)
    conclusion = LinearConclusion(target, ">=", m, other, b)

    return LinearConjecture(hypothesis, conclusion, symbol, touch)

def make_all_upper_linear_conjectures(df, target, others, properties):
    conjectures = []
    for other in others:
        for k in range(4):  # Considering hypotheses of none, one, two, and three boolean properties
            for prop_comb in combinations(properties, k):
                if other != target:
                    conjecture = make_upper_linear_conjecture(df, target, other, prop_comb)
                    if conjecture:
                        conjectures.append(conjecture)
    return conjectures

def make_all_lower_linear_conjectures(df, target, others, properties):
    conjectures = []
    for other in others:
        for k in range(4):  # Considering hypotheses of none, one, two, and three boolean properties
            for prop_comb in combinations(properties, k):
                if other != target:
                    conjecture = make_lower_linear_conjecture(df, target, other, prop_comb)
                    if conjecture:
                        conjectures.append(conjecture)
    return conjectures

def sort_by_touch_number(conjectures):
    return sorted(conjectures, key=lambda x: x.touch, reverse=True)

def apply_theo_heuristic(conjectures):
    filtered_conjectures = []
    for conj_1 in conjectures:
        is_general = True
        for conj_2 in filtered_conjectures:
            if (conj_1.conclusion.slope == conj_2.conclusion.slope and
                conj_1.conclusion.intercept == conj_2.conclusion.intercept and
                conj_1.conclusion.inequality == conj_2.conclusion.inequality and
                set(conj_1.hypothesis.statements).issubset(set(conj_2.hypothesis.statements))):
                is_general = False
                break
        if is_general:
            filtered_conjectures.append(conj_1)
    return filtered_conjectures

def apply_static_dalmatian_heuristic(df, conjectures):
    filtered_conjectures = []
    for conj in conjectures:
        conj_distances = conj.calculate_distances(df)
        keep_conj = True
        for other_conj in filtered_conjectures:
            other_distances = other_conj.calculate_distances(df)
            if np.all(conj_distances >= other_distances):
                keep_conj = False
                break
        if keep_conj:
            filtered_conjectures.append(conj)
    return filtered_conjectures

def txgraffiti_conjecture_generation(df, targets, invariants, properties):
    conjectures = []
    for target in targets:
        upper_conjectures = make_all_upper_linear_conjectures(df, target, invariants, properties)
        lower_conjectures = make_all_lower_linear_conjectures(df, target, invariants, properties)
        conjectures += upper_conjectures + lower_conjectures

    conjectures = sort_by_touch_number(conjectures)
    conjectures = apply_theo_heuristic(conjectures)
    conjectures = apply_static_dalmatian_heuristic(df, conjectures)

    return conjectures

# Generate synthetic data for scale-free graphs
n_samples = 1_000
np.random.seed(42)

# Generate scale-free graphs and convert them to simple graphs
graphs = [nx.Graph(nx.scale_free_graph(np.random.randint(10, 30))) for _ in range(n_samples)]

# Define properties
def avg_clustering_coefficient(G):
    return nx.average_clustering(G)

def degree_assortativity_coefficient(G):
    return nx.degree_assortativity_coefficient(G)

def number_of_triangles(G):
    return sum(nx.triangles(G).values()) // 3

def max_degree(G):
    return max(dict(G.degree()).values())

def average_shortest_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    return np.nan

# Generate properties
avg_clustering = np.array([avg_clustering_coefficient(G) for G in graphs])
degree_assortativity = np.array([degree_assortativity_coefficient(G) for G in graphs])
num_triangles = np.array([number_of_triangles(G) for G in graphs])
max_degree_values = np.array([max_degree(G) for G in graphs])
avg_shortest_path = np.array([average_shortest_path_length(G) for G in graphs])

# Create the DataFrame
data = {
    'avg_clustering': avg_clustering,
    'degree_assortativity': degree_assortativity,
    'num_triangles': num_triangles,
    'max_degree': max_degree_values,
    'avg_shortest_path': avg_shortest_path
}
df = pd.DataFrame(data).dropna()  # Drop rows with NaN values

# Define the targets, invariants, and properties
targets = ["avg_clustering", "degree_assortativity", "num_triangles", "max_degree", "avg_shortest_path"]
invariants = ["avg_clustering", "degree_assortativity", "num_triangles", "max_degree", "avg_shortest_path"]
properties = []

# Generate conjectures using the TxGraffiti algorithm
conjectures = txgraffiti_conjecture_generation(df, targets, invariants, properties)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/AI-discovery-in-mathematics-with-TxGraffiti/env/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/5fd3a57a2ff14938a57fb9137eee158d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/5fd3a57a2ff14938a57fb9137eee158d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 952 COLUMNS
At line 2849 RHS
At line 3797 BOUNDS
At line 3800 ENDATA
Problem MODEL has 947 rows, 2 columns and 1894 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 941 (-6) rows, 2 (0) columns and 1882 (-12) elements
0  Obj 0 Primal inf 254.42235 (941) Dual inf 22.091754 (2) w.o. free dual inf (0)
2  Obj 512.73286
2  Obj -7.9799931e+11 Primal inf 6.9791851e+12 (522) Dual inf 4.80356

In [2]:
# Print the generated conjectures
for i, conj in enumerate(conjectures[:20]):
    print(f"Conjecture {i+1}. ", conj, "\n")

Conjecture 1.  For any scale-free graph G, num_triangles(G) <= -4/5*max_degree(G) + 153/5, with equality on 3 instances. 

Conjecture 2.  For any scale-free graph G, num_triangles(G) >= 1/6*max_degree(G) + -1/2, with equality on 3 instances. 

Conjecture 3.  For any scale-free graph G, max_degree(G) <= 1/7*num_triangles(G) + 141/7, with equality on 2 instances. 

Conjecture 4.  For any scale-free graph G, avg_clustering(G) <= -2/7*degree_assortativity(G) + 1/4, with equality on 0 instances. 

Conjecture 5.  For any scale-free graph G, avg_clustering(G) <= -1/7*avg_shortest_path(G) + 7/10, with equality on 0 instances. 

Conjecture 6.  For any scale-free graph G, avg_clustering(G) >= 1/10*avg_shortest_path(G) + -1/10, with equality on 0 instances. 

Conjecture 7.  For any scale-free graph G, degree_assortativity(G) <= -1/3*avg_clustering(G) + 0, with equality on 0 instances. 

Conjecture 8.  For any scale-free graph G, degree_assortativity(G) <= 5/8*avg_shortest_path(G) + -3/2, with equ